In [1]:
import json, re,  os, time, random, time
from bs4 import BeautifulSoup   
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC   
from datetime import datetime, date, timedelta
import pymongo
import logging  
from lib.jira import jiradriver
# ChromeDriverManager().install()

In [2]:
jira = jiradriver().init() 

driver_path: C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\bin\Debug\net48\Chrome\130.0.6723.69\X64\chromedriver.exe


In [3]:
jira = jira.get_issues('SCuBA').log(time_spent='60m')
_= jira.init() 

In [5]:
_= jira.init() 

In [4]:
jira.get_issues('SCuBA').subo() 

In [12]:
jira.driver.quit()

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.jira
collection = db.issues 
db.issues.create_index('href', unique = True)



df = pd.DataFrame(list(collection.find()))
df['tnum'] = df['href'].apply(lambda s: s.split('/').pop())
df.sort_index(inplace=True, ascending=False)
df['tnum'] + ' ' + df['title'] 

In [ ]:
posts
df = pd.DataFrame(posts)

In [ ]:
dct = {'driver_path':'xxx'}
if 'driver_path' in dct.keys():
    print('y')
if 'driver_paths' in dct.keys():
    print('y')    

In [ ]:
options = Options() 
options.add_argument("--window-size=1220,980")  
options.add_argument('--log-level=3')
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install() ,options=options) 
driver.get(f"https://accounts.craigslist.org/login")  
driver.find_element(By.XPATH, '//input[contains(@id, "Email")]').send_keys('timkopp@gmail.com')
driver.find_element(By.XPATH, '//input[contains(@id, "Password")]').send_keys(ctx.config['clpass']) 
driver.find_element(By.XPATH, '//*[contains(@id, "login")]').click()
time.sleep(2)   
renew = driver.find_elements(By.XPATH,"//form[contains(@action,'7739609697') and contains(@class, 'renew') ]//input[@type='submit']")
if len(renew) > 0:
    #renew[0].click()
    #driver.quit()
    pass